# 2010-era to 2020-era census tracts mapping

This notebook takes in the geometry of the 2010-era and 2020-era census tracts and generates a lookup table between them. 
Most census tracts have a one-to-one relationship. If a 2020 census tract has multiple related 2010 census tracts, the largest is assumed.

In [1]:
import geopandas as gpd
import pandas as pd

In [19]:
# overlaying 2010-era census tract geometry onto 2020-era
original_geom = gpd.read_file("../gis/equity_analysis_zones_2019.geojson")
new_geom = gpd.read_file("../gis/equity_analysis_zones_2020.geojson")
gdf = gpd.overlay(new_geom, original_geom, how='intersection', keep_geom_type=True)

/Users/charliehenry/anaconda3/envs/atd/lib/python3.9/site-packages/geopandas/geodataframe.py:1815: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  merged_geom = block.unary_union


In [20]:
a = pd.DataFrame(gdf.groupby("GEOID_1")["NAME_1"].count())
a = a[a["NAME_1"] > 1]
a = a.sort_values("NAME_1", ascending=False)

In [21]:
# for those overlapping with 2 or more 2020 census tracts, we just assume that the largest (in area) represents the 2019 zone.
data = []
for index, row in a.iterrows():
    b = gdf[gdf["GEOID_1"] == index]
    b["area"] = b.area
    b = b.sort_values(by="area", ascending=False)
    max = b["area"].iloc[0]
    data.append({"2019_tract": b.iloc[0]["GEOID_2"], "2020_tract": b.iloc[0]["GEOID_1"]})

/var/folders/ym/cg59ddhx7jl408c4hs61x_wm0000gp/T/ipykernel_26558/2858818050.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  b["area"] = b.area
/Users/charliehenry/anaconda3/envs/atd/lib/python3.9/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/var/folders/ym/cg59ddhx7jl408c4hs61x_wm0000gp/T/ipykernel_26558/2858818050.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  b["area"] = b.area
/Users/char

In [22]:
# handling 1 to 1 merges
a = pd.DataFrame(gdf.groupby("GEOID_1")["NAME_1"].count())
a = a[a["NAME_1"] == 1]
a = a.sort_values("NAME_1", ascending=False)

for index, row in a.iterrows():
    b = gdf[gdf["GEOID_1"] == index]
    data.append({"2019_tract": b.iloc[0]["GEOID_2"], "2020_tract": b.iloc[0]["GEOID_1"]})

In [23]:
pd.DataFrame(data).to_csv("2019_to_2020.csv")